In [1]:
import pandas
import ast
import torch
from torch.utils.data import Dataset


class Articles(Dataset):
    def __init__(self, filename, classes):
        self.dir = dir
        self.classes = classes
        self.df = pandas.read_csv(filename, delimiter=",", encoding="utf-8")

    def __getitem__(self, index):
        text = torch.LongTensor(ast.literal_eval(self.df["text"][index]))
        # on a qu'une seule classe dans le dataset
        # labels = ast.literal_eval(self.df["topic"][index])
        # topic = torch.zeros(self.classes)
        # for label in labels:
        #     topic[label] = 1
        topic = torch.LongTensor([self.df["topic"][index]])
        return text, topic

    def __len__(self):
        return len(self.df)

In [2]:
import torch


class TopicClassifier(torch.nn.Module):
    def __init__(self, tokens, classes):
        super(TopicClassifier, self).__init__()
        device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
        self.embeding = torch.nn.Embedding(tokens, 100)
        self.lstm = torch.nn.LSTM(100, 1024, batch_first=True)
        # replace (1, 1, 512) with (1, 50, 512) for prediction
        self.cell = (torch.zeros(1, 1, 1024).to(device),
                     torch.zeros(1, 1, 1024).to(device))
        self.fc = torch.nn.Linear(1024, classes)

    # dans le dataset un article n'appartient qu'à un seul topic, nous allons modifier le modele pour correspondre à cela
    def forward(self, x):
        x = self.embeding(x)
        l, self.cell = self.lstm(x, self.cell)
        l = l[:, -1, :]
        x = self.fc(l)
        # return torch.sigmoid(x)
        return x
